In [2]:
import os
import numpy as np
import pandas as pd
from loaders.loading import load_train
from transformers.dash_transformer import DashTransformer
from transformers.dict_nbhd_transformer import DictNBHDTransformer
from transformers.dict_transformer import DictTransformer
from transformers.digit_transformer import DigitTransformer
from transformers.flat_transformer import FlatTransformer
from transformers.dict_class_transformer import DictClassTransformer
from sklearn.pipeline import Pipeline
from transformers.letters_transformer import LettersTransformer
from transformers.scored_chain import ScoredChain
from transformers.self_transformer import SelfTransformer
from time import gmtime, strftime
from transformers.simple_transliterator import SimpleTransliterator

In [3]:
df = load_train(['class', 'before', 'after'])
df['before_prev'] = df['before'].shift(1)
df['before_next'] = df['before'].shift(-1)
df = df.fillna('')
print(df.info())

classes = ['PLAIN', 'DATE', 'PUNCT', 'ORDINAL', 'VERBATIM', 'LETTERS', 'CARDINAL', 'MEASURE', 'TELEPHONE', 'ELECTRONIC',
           'DECIMAL', 'DIGIT', 'FRACTION', 'MONEY', 'TIME']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10574516 entries, 0 to 10574515
Data columns (total 5 columns):
class          object
before         object
after          object
before_prev    object
before_next    object
dtypes: object(5)
memory usage: 403.4+ MB
None


In [4]:
test_size = int(len(df.index)*0.1)
x_train = df.drop(['after'], axis=1).head(len(df.index)-test_size)
y_train = df['after'].head(len(df.index)-test_size)
x_test = df.drop(['after'], axis=1).tail(test_size)
y_test = df['after'].tail(test_size)

In [5]:
transform_chain = ScoredChain([
    ('dash', DashTransformer()),
    ('digit', DigitTransformer()),
    ('punсt', DictClassTransformer(u'PUNCT', 1.0)),
    ('verbatim', DictClassTransformer(u'VERBATIM', 1.0)),
    ('self', SelfTransformer(threshold=0.5, modelpath='models/self.model.train')),
    ('dict_nbhd', DictNBHDTransformer(0.5)),
    ('letters', LettersTransformer()),
    # ('dict', DictTransformer(0.5)),
    ('translit', SimpleTransliterator('cyrtranslit')),
    # ('flat', FlatTransformer())
], metrics=[
    ('tp', lambda X: np.mean((X["after"] == y_test))),
    ('fp', lambda X: np.mean((X["after"].notnull()) & ~(X["after"] == y_test)))
])
predicted_val = transform_chain.fit(x_train, y_train)
predicted = transform_chain.transform(x_test)
tp = np.mean((predicted["after"] == y_test))
fp = np.mean((predicted["after"].notnull()) & ~(predicted["after"] == y_test))
print(f'pipeline test TP {tp}')
print(f'pipeline test FP {fp}')

MorphologyExtractor create index: 0it [00:00, ?it/s]

MorphologyExtractor create index: 12560it [00:00, 125565.99it/s]

MorphologyExtractor create index: 22604it [00:00, 113003.54it/s]

MorphologyExtractor create index: 36331it [00:00, 121086.80it/s]

MorphologyExtractor create index: 44405it [00:00, 107175.80it/s]

...

DictNBHDTransformer fit stage 1:  63%|██████▎   | 6027981/9517065 [00:53<00:31, 111994.35it/s]

DictNBHDTransformer fit stage 1:  64%|██████▎   | 6044966/9517065 [00:53<00:30, 112073.69it/s]

DictNBHDTransformer fit stage 1:  64%|██████▎   | 6061347/9517065 [00:54<00:30, 112141.88it/s]

DictNBHDTransformer fit stage 1:  64%|██████▍   | 6077152/9517065 [00:55<00:31, 109105.13it/s]

DictNBHDTransformer fit stage 1:  64%|██████▍   | 6093389/9517065 [00:55<00:31, 109200.58it/s]

In [7]:
predicted[(predicted["after"] == y_test)]['class'].value_counts()

PLAIN         699728
PUNCT         229395
LETTERS        18051
CARDINAL       15800
VERBATIM       15627
DATE            8484
ORDINAL         2294
MEASURE         1152
DIGIT            183
ELECTRONIC       169
TIME             151
TELEPHONE        130
DECIMAL           72
FRACTION          29
MONEY              8
Name: class, dtype: int64

In [8]:
predicted[~(predicted["after"] == y_test) & ~(predicted["after"].isnull())]['class'].value_counts()

PLAIN         33842
CARDINAL        767
ELECTRONIC      380
LETTERS         344
ORDINAL         300
DATE            191
MEASURE          85
VERBATIM         41
MONEY            27
TELEPHONE        12
TIME              4
DECIMAL           1
Name: class, dtype: int64

In [9]:
predicted[(predicted["after"].isnull())]['class'].value_counts()

CARDINAL      11141
DATE          10324
MEASURE        2688
ORDINAL        2247
PLAIN           940
TELEPHONE       861
DECIMAL         660
LETTERS         639
MONEY           252
FRACTION        228
TIME            128
ELECTRONIC       76
Name: class, dtype: int64

In [16]:
predicted[~(predicted["after"] == y_test) & ~(predicted["after"].isnull()) & (predicted['class']=='PLAIN')].assign(after_true=y_test).sample(20)

,class,before,before_prev,before_next,after,after_true
9754191,PLAIN,and,Religion,Law,а_trans н_trans д_trans,э_trans н_trans д_trans
9809433,PLAIN,Amorpha,—,L.,а_trans м_trans о_trans р_trans п_trans х_tran...,а_trans м_trans о_trans р_trans ф_trans а_trans
10420900,PLAIN,Biology,Avian,34,б_trans и_trans о_trans л_trans о_trans г_tran...,б_trans и_trans о_trans л_trans о_trans д_tran...
10256172,PLAIN,X,-,о,x,x_latin
10358267,PLAIN,Moje,«,3,м_trans о_trans ж_trans е_trans,м_trans о_trans х_trans
9856680,PLAIN,Que,Dicen,Soy,q_trans у_trans е_trans,к_trans в_trans и_trans
9708292,PLAIN,Mullins,Rich,TimeLine,м_trans у_trans л_trans л_trans и_trans н_tran...,м_trans а_trans л_trans л_trans и_trans н_tran...
10042255,PLAIN,Basketball,States,Writers,б_trans а_trans с_trans к_trans е_trans т_tran...,б_trans а_trans с_trans к_trans е_trans т_tran...
10160035,PLAIN,Raphaelites,-,from,р_trans а_trans п_trans х_trans а_trans е_tran...,р_trans а_trans ф_trans а_trans э_trans л_tran...
9852123,PLAIN,F,истребитель,-15,f,f_latin


In [50]:
df[~(df['before'].str.match('^-?\d*\s?\d*\s?\/\s?\d+\s?\d*$')) & (df['class'] == 'FRACTION')]

,sentence_id,token_id,class,before,after,before_prev,before_next
40420,2962,7,FRACTION,7½,семи целых и одной второй,около,фунтов
157446,11425,6,FRACTION,3½,три целых и одну вторую,на,очка
445580,32294,9,FRACTION,59½,пятидесяти девяти целых и одной второй,139 976 рублей,копейки
765884,55575,1,FRACTION,¼,одна четвертая,второй,финалдата
803808,58316,6,FRACTION,½,одна вторая,номиналом,шиллинга
999135,72597,18,FRACTION,⅔,две третьих,на,территории
1253476,91132,28,FRACTION,½,одна вторая,cra,gut
1584453,115013,1,FRACTION,8½,восемь целых и одна вторая,«,женщин
1584466,115013,14,FRACTION,8½,восемь целых и одна вторая,.,women
1876892,136047,6,FRACTION,½,одна вторая,в,","
